# CP

In this example, we are going to first create a large sparse tensor and decompose it with a sparse CANDECOMP/PARAFAC decomposition.

In [2]:
import numpy as np
import sparse
import tensorly as tl

In [3]:
shape = (1000, 1001, 1002, 100)
rank = 5

starting_factors = [sparse.random((i, rank)) for i in shape]
starting_factors
starting_weights = sparse.ones(rank)

Now convert it to a tensor. It is very important to use `kruskal_to_tensor` from the sparse backend, as a fully dense version of the tensor would use several TB of memory.

In [4]:
from tensorly.contrib.sparse.cp_tensor import cp_to_tensor
tensor = cp_to_tensor((starting_weights, starting_factors))
tensor

Format,coo
Data Type,float64
Shape,"(1000, 1001, 1002, 100)"
nnz,3708
Density,3.696901900494715e-08
Read-only,True
Size,144.8K
Storage ratio,0.0


As before, we can compare the actual spase used by the tensor vs. what it would require if it were dense.

In [5]:
tensor.nbytes / 1e9                # Actual memory usage in GB

0.00014832

In [6]:
np.prod(tensor.shape) * 8 / 1e9    # Memory usage if array was dense, in GB

802.4016

In [7]:
import time
%load_ext memory_profiler

Note that even though we started with a sparse tensor, the factors are dense. This is because we used the dense version of `parafac`. Since the factors are in general dense, even for a sparse tensor, this is generally preferred. 

Let's decompose the sparse tensor into a sparse Kruskal tensor:

In [8]:
from tensorly.contrib.sparse.decomposition import parafac as parafac_sparse

In [9]:
%%memit
start_time = time.time()
sparse_kruskal = parafac_sparse(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

reconstruction error=0.24878769515229152
iteration 1, reconstruction error: 0.02308228930031955, decrease = 0.22570540585197196, unnormalized = 0.16101102424357264
iteration 2, reconstruction error: 0.00022784518494734887, decrease = 0.0228544441153722, unnormalized = 0.0015893391734255321
iteration 3, reconstruction error: 6.074141378714362e-06, decrease = 0.00022177104356863451, unnormalized = 4.237030877060909e-05
iteration 4, reconstruction error: 2.036467760587666e-07, decrease = 5.8704946026555955e-06, unnormalized = 1.4205426320806723e-06
iteration 5, reconstruction error: 0.0, decrease = 2.036467760587666e-07, unnormalized = 0.0
iteration 6, reconstruction error: 0.0, decrease = 0.0, unnormalized = 0.0
PARAFAC converged after 6 iterations
Took 4 mins 52 secs
peak memory: 1693.52 MiB, increment: 1551.77 MiB


Let's look at the result

In [10]:
sparse_kruskal

(weights, factors) : rank-5 CPTensor of shape (1000, 1001, 1002, 100) 

Because the `factors_sparse` are sparse, we can reconstruct them into a tensor without using too much memory. In general, this will not be the case, but it is for our toy example. Let's do this to look at the absolute error for the decomposition. 

In [11]:
tl.norm(tensor - cp_to_tensor(sparse_kruskal))

8.076887302831362e-09

It is not actually necessary to compute this, as the same as the norm of the tensor times the reconstruction error that was printed by the algorithm (you can pass `return_errors=True` to `parafac()` to have the reconstruction errors be returned along with the factors). That is, $$\mathrm{reconstruction\ error} = \frac{\|\mathrm{tensor} - \mathrm{kruskal\_to\_tensor}(\mathrm{factors})\|_2}{\|\mathrm{tensor}\|_2}$$ (they won't be exactly the same due to numerical differences in how they are calculated).

In [12]:
tl.norm(tensor - cp_to_tensor(sparse_kruskal))/tl.norm(tensor)

1.1578899658945145e-09

Let's look at one of the nonzero entries to see how close it is to the original tensor. The factors satisfy $$\sum_{r=0}^{R-1} {f_0}_r\circ {f_1}_r \circ {f_2}_r \circ {f_3}_r,$$ where $R$ is the rank (here 5), ${f_i}_r$ is the $r$-th column of the $i$-th factor of the decomposition, and $\circ$ is the vector outer product. Component-wise, this translates to a product of corresponding elements per component for each factor, summed over the columns.

In [13]:
tensor.coords

array([[ 65,  65,  65, ..., 924, 924, 924],
       [ 91,  91,  91, ..., 916, 916, 916],
       [ 59,  59,  67, ..., 574, 620, 831],
       [ 23,  34,  23, ...,  60,  60,  60]])

In [14]:
orig_val = tensor[tuple(tensor.coords.T[0])]
orig_val

0.2810493285343943

In [15]:
weights_sparse, factors_sparse = sparse_kruskal
sparse_val = np.sum(np.prod(sparse.stack([factors_sparse[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
sparse_val

0.2810493285347173

In [16]:
np.abs(orig_val - sparse_val)

3.230193890146893e-13

The difference here is mostly due to random chance. The total reconstruction errors for the two runs of algorithm were roughly the same. In general, the error of the factorization will vary due to the randomness of the initial factors chosen by the algorithm.